In [1]:
import pandas as pd
# 1. Load the CSV file into a pandas dataframe
csv_file_path = 'IPL MATCH.csv'
df = pd.read_csv(csv_file_path)
df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2008,Bangalore,18-04-2008,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,223,20,N,NaN,Asad Rauf,RE Koertzen
1,335983,2008,Chandigarh,19-04-2008,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,241,20,N,NaN,MR Benson,SL Shastri
2,335984,2008,Delhi,19-04-2008,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,130,20,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2008,Mumbai,20-04-2008,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,166,20,N,NaN,SJ Davis,DJ Harper
4,335986,2008,Kolkata,20-04-2008,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,111,20,N,NaN,BF Bowden,K Hariharan


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# pip install mysql-connector
import mysql.connector

# Connect to MySQL server (replace with your own credentials)
db_connection = mysql.connector.connect(
        host='localhost',  # Change to your MySQL host
        user='root',       # Change to your MySQL username
        password='Azsxdcf123@',  # Change to your MySQL password
    )

# Create a cursor object to interact with the database
cursor = db_connection.cursor()

## createdata base and table

In [ ]:

# Step 1: Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS cricket_matches")

# Step 2: Use the newly created database
cursor.execute("USE cricket_matches")

# Step 3: Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS match_details (
    id INT NOT NULL,
    season INT,
    city VARCHAR(255),
    date DATE,
    match_type VARCHAR(50),
    player_of_match VARCHAR(255),
    venue VARCHAR(255),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    toss_winner VARCHAR(255),
    toss_decision VARCHAR(50),
    winner VARCHAR(255),
    result VARCHAR(50),
    result_margin INT,
    target_runs INT,
    target_overs INT,
    super_over CHAR(1),
    method VARCHAR(50),
    umpire1 VARCHAR(255),
    umpire2 VARCHAR(255),
    PRIMARY KEY (id)
)
"""

# Execute the query to create the table
cursor.execute(create_table_query)

print("Database and table created successfully.")


In [ ]:
df.info()
df.dtypes
len(df.columns)

### import all data into mysql database

In [ ]:
# Convert 'date' column to proper Date format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y').dt.date

# Define the MySQL insert query
insert_query = """
INSERT INTO match_details (
    id, season, city, date, match_type, player_of_match, venue, team1, team2, toss_winner, toss_decision, 
    winner, result, result_margin, target_runs, target_overs, super_over, method, umpire1, umpire2
) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
"""

# Prepare the data as tuples
# Exclude the index by setting index=False in itertuples()
data_tuples = list(df.itertuples(index=False, name=None))

# Ensure that the number of columns in data_tuples matches the number of placeholders in insert_query
print(f"Number of columns in data_tuples: {len(data_tuples[0])}")
print(f"Number of placeholders in insert_query: {insert_query.count('%s')}")

# Insert all data into the table
cursor.executemany(insert_query, data_tuples)

# Commit the transaction
db_connection.commit()
print("Data inserted successfully.")


### Load the data from table

In [ ]:

# Define the query to fetch all data from the table 'match_details'
query = "SELECT * FROM match_details"

# Execute the query
cursor.execute(query)

# Fetch all rows from the result set
data = cursor.fetchall()

# Get the column names
columns = [desc[0] for desc in cursor.description]

# Load data into pandas DataFrame (df2)
df2 = pd.DataFrame(data, columns=columns)
df2.head()

#### Clean the data

In [ ]:
len(df2)

In [ ]:
df2.isnull().sum()

In [ ]:
df2.drop("method", inplace=True,axis=1)


In [ ]:
# Example: Convert 'date' column to datetime if needed
df2['date'] = pd.to_datetime(df2['date'], errors='coerce')


In [ ]:
# Replace null values in 'city' with the first word from 'venue'
df2['city'] = df2['city'].fillna(df2['venue'].str.split().str[0])

# Show the DataFrame after the replacement
print(df2[['city', 'venue']].head())  # Display the first few rows for verification


In [ ]:
# Show rows with null values in a specific column
null_rows_column = df2[["city", "venue"]][df2['city'].isnull()]

# Print the rows with null values in that column
print(null_rows_column)


In [ ]:
df2[df2["city"]== "Dubai"]

In [ ]:
df2.isnull().sum()

In [ ]:
df2.head()

In [ ]:
df2['date'] = pd.to_datetime(df2['date'], errors='coerce').dt.strftime('%Y-%m-%d')

df.dtypes

### again store in database

In [ ]:

# Step 3: Create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS clean_match_details (
    id INT NOT NULL,
    season INT,
    city VARCHAR(255),
    date DATE,
    match_type VARCHAR(50),
    player_of_match VARCHAR(255),
    venue VARCHAR(255),
    team1 VARCHAR(255),
    team2 VARCHAR(255),
    toss_winner VARCHAR(255),
    toss_decision VARCHAR(50),
    winner VARCHAR(255),
    result VARCHAR(50),
    result_margin INT,
    target_runs INT,
    target_overs INT,
    super_over CHAR(1),
    umpire1 VARCHAR(255),
    umpire2 VARCHAR(255),
    PRIMARY KEY (id)
)
"""

# Execute the query to create the table
cursor.execute(create_table_query)


In [ ]:
# Define the MySQL insert query (without the 'method' column)
insert_query = """
INSERT INTO clean_match_details (
    id, season, city, date, match_type, player_of_match, venue, team1, team2, toss_winner, toss_decision, 
    winner, result, result_margin, target_runs, target_overs, super_over, umpire1, umpire2
) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
"""

# Prepare the data as tuples, excluding the index
data_tuples = list(df2.itertuples(index=False, name=None))

# Insert all data into the table
cursor.executemany(insert_query, data_tuples)

# Commit the transaction
db_connection.commit()


print("Cleaned data (method column dropped) inserted successfully into clean_data table.")


In [65]:
def sql_query(query):
    # Execute the query
    cursor.execute(query)

    # Fetch all rows from the result set
    data = cursor.fetchall()

    # Get the column names
    columns = [desc[0] for desc in cursor.description]

    # Load data into pandas DataFrame (df2)
    df2 = pd.DataFrame(data, columns=columns)
    return df2

In [67]:
# 
sql_query("select *from clean_match_details").head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,umpire1,umpire2
0,335982,2008,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140,223,20,N,Asad Rauf,RE Koertzen
1,335983,2008,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33,241,20,N,MR Benson,SL Shastri
2,335984,2008,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9,130,20,N,Aleem Dar,GA Pratapkumar
3,335985,2008,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5,166,20,N,SJ Davis,DJ Harper
4,335986,2008,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5,111,20,N,BF Bowden,K Hariharan


In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# 1. Load the CSV file into a pandas dataframe
csv_file_path = 'CWC2023.csv'
df = pd.read_csv(csv_file_path)
# df.head()
# df = df.applymap(lambda x: str(x).replace(',','.') if isinstance(x, float) else x)

# df = df.apply(pd.to_numeric, errors="ignore")
# 2. Convert 'match_date' column to 'YYYY-MM-DD' format with error handling
# This will convert invalid dates to NaT (Not a Time) and allow us to deal with them.
df['Match Date'] = pd.to_datetime(df['Match Date'], format='%d-%m-%Y', errors='coerce')

# Optionally, print out rows where the conversion failed to identify problematic dates
invalid_dates = df[df['Match Date'].isna()]
if not invalid_dates.empty:
    print("Invalid dates found in the following rows:")
    print(invalid_dates)

# 3. Drop rows with invalid dates (if any), or you can choose to handle them differently
df.dropna(subset=['Match Date'], inplace=True)

# Ensure that date and time columns are properly formatted
df['Match Date'] = pd.to_datetime(df['Match Date'], format='%d-%m-%Y', errors='coerce').dt.date
df['Match Time'] = pd.to_datetime(df['Match Time'], format='%I:%M %p', errors='coerce').dt.time

# Handle invalid date/time by replacing NaT with None (which will be treated as NULL in MySQL)
df['Match Date'] = df['Match Date'].where(df['Match Date'].notna(), None)
df['Match Time'] = df['Match Time'].where(df['Match Time'].notna(), None)


df.head(20)

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()
df.dropna(inplace= True)
df.isnull().sum()

In [ ]:

# 4. Convert 'Runrate A' and 'Runrate B' columns to numeric, replacing commas with dots (if necessary)
df['Runrate A'] = df['Runrate A'].str.replace(',', '.').astype(float)
df['Runrate B'] = df['Runrate B'].str.replace(',', '.').astype(float)
df

In [ ]:
for i, row in df.iterrows():
    
    print(i,row)
 

In [ ]:
# 2. Establish connection to MySQL (use your actual credentials)
try:
    connection = mysql.connector.connect(
        host='localhost',  # Change to your MySQL host
        user='root',       # Change to your MySQL username
        password='Azsxdcf123@',  # Change to your MySQL password
    )
    
    if connection.is_connected():
        print("Connected to MySQL")

        # 3. Create a new database if it does not exist
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS cricket_data")
        cursor.execute("USE cricket_data")

        # 4. Create a table if it does not exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS match_stats (
            match_id INT PRIMARY KEY,
            match_date DATE,
            match_time TIME,
            city VARCHAR(100),
            stadium VARCHAR(100),
            team_a VARCHAR(50),
            team_b VARCHAR(50),
            toss_winner VARCHAR(50),
            toss_decision VARCHAR(50),
            score_a INT,
            wickets_a INT,
            overs_played_a DECIMAL(4,1),
            maiden_overs_a INT,
            fours_a INT,
            sixes_a INT,
            boundaries_a INT,
            runrate_a DECIMAL(10,2),
            leg_byes_a INT,
            byes_a INT,
            wides_a INT,
            no_balls_a INT,
            penalty_a INT,
            extras_a INT,
            score_b INT,
            wickets_b INT,
            overs_played_b DECIMAL(20,1),
            maiden_overs_b INT,
            fours_b INT,
            sixes_b INT,
            boundaries_b INT,
            runrate_b DECIMAL(10,2),
            leg_byes_b INT,
            byes_b INT,
            wides_b INT,
            no_balls_b INT,
            penalty_b INT,
            extras_b INT,
            winning_team VARCHAR(50),
            margin VARCHAR(50),
            man_of_the_match VARCHAR(100),
            umpire_1 VARCHAR(100),
            umpire_2 VARCHAR(100),
            tv_umpire VARCHAR(100),
            match_referee VARCHAR(100)
        );
        """
        cursor.execute(create_table_query)

        # 5. Insert data into the table
        for i, row in df.iterrows():
            insert_query = """
            INSERT INTO match_stats (
                match_id, match_date, match_time, city, stadium, team_a, team_b, toss_winner, toss_decision,
                score_a, wickets_a, overs_played_a, maiden_overs_a, fours_a, sixes_a, boundaries_a, runrate_a,
                leg_byes_a, byes_a, wides_a, no_balls_a, penalty_a, extras_a, score_b, wickets_b, overs_played_b,
                maiden_overs_b, fours_b, sixes_b, boundaries_b, runrate_b, leg_byes_b, byes_b, wides_b, no_balls_b,
                penalty_b, extras_b, winning_team, margin, man_of_the_match, umpire_1, umpire_2, tv_umpire, match_referee
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                      %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """
            
            cursor.execute(insert_query, tuple(row))
        
        connection.commit()
        print("Data inserted successfully!")

except Error as e:
    print(f"Error: {e}")
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


In [ ]:

# Optionally, if you want to visualize or analyze the data further using pandas or matplotlib:
import matplotlib.pyplot as plt

# Example: Plot a simple chart of match scores
plt.figure(figsize=(10,5))
df['Score A'].sort_values().plot(kind='bar', title="Scores of Team A")
plt.xlabel('Match ID')
plt.ylabel('Score')
plt.show()


In [ ]:
df.head()

In [ ]:
df.head()
df.value_counts(["Stadium"])

In [ ]:
df[["Stadium","City","Score A"]].sort_values(by="Score A")

In [ ]:
pip install sqlalchemy

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Table, MetaData, Column, Integer, String, Date, Time, DECIMAL

# Load CSV into DataFrame
csv_file_path = 'your_csv_file.csv'  # Adjust this path to the correct location of your CSV
columns = [
    'Match ID', 'Match Date', 'Match Time', 'City', 'Stadium', 'Team A', 'Team B', 'Toss Winner', 'Toss Decision', 
    'Score A', 'Wickets A', 'Overs Played A', 'Maiden Overs A', '4s A', '6s A', 'Boundaries A', 'Runrate A', 
    'Leg Byes A', 'Byes A', 'Wides A', 'No Balls A', 'Penalty A', 'Extras A', 'Score B', 'Wickets B', 'Overs Played B', 
    'Maiden Overs B', '4s B', '6s B', 'Boundaries B', 'Runrate B', 'Leg Byes B', 'Byes B', 'Wides B', 'No Balls B', 
    'Penalty B', 'Extras B', 'Wining Team', 'Margin', 'Man of the Match', 'Umpire 1', 'Umpire 2', 'TV Umpire', 'Match Refree'
]
df = pd.read_csv(csv_file_path, names=columns, header=0)

# Ensure correct data types and formats for 'Match Date' and 'Match Time'
df['Match Date'] = pd.to_datetime(df['Match Date'], format='%d-%m-%Y', errors='coerce').dt.date
df['Match Time'] = pd.to_datetime(df['Match Time'], format='%I:%M %p', errors='coerce').dt.time

# Replace commas with periods for decimal values (e.g., 'Runrate A' and 'Runrate B')
df['Runrate A'] = pd.to_numeric(df['Runrate A'].str.replace(',', '.'), errors='coerce')
df['Runrate B'] = pd.to_numeric(df['Runrate B'].str.replace(',', '.'), errors='coerce')

# Handle invalid dates and times
df['Match Date'] = df['Match Date'].where(df['Match Date'].notna(), None)
df['Match Time'] = df['Match Time'].where(df['Match Time'].notna(), None)

# Define the connection string
# Replace with your actual credentials
username = 'root'
password = 'Azsxdcf123@'
database = 'cricket_data'
host = 'localhost'  # Or your MySQL host, e.g., '127.0.0.1'

# Create SQLAlchemy engine for MySQL
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

# Create a connection to the database
connection = engine.connect()

# Create table schema (using SQLAlchemy)
metadata = MetaData()

match_stats_table = Table('match_stats', metadata,
    Column('match_id', Integer, primary_key=True),
    Column('match_date', Date),
    Column('match_time', Time),
    Column('city', String(100)),
    Column('stadium', String(100)),
    Column('team_a', String(50)),
    Column('team_b', String(50)),
    Column('toss_winner', String(50)),
    Column('toss_decision', String(50)),
    Column('score_a', Integer),
    Column('wickets_a', Integer),
    Column('overs_played_a', DECIMAL(4, 1)),
    Column('maiden_overs_a', Integer),
    Column('fours_a', Integer),
    Column('sixes_a', Integer),
    Column('boundaries_a', Integer),
    Column('runrate_a', DECIMAL(5, 2)),
    Column('leg_byes_a', Integer),
    Column('byes_a', Integer),
    Column('wides_a', Integer),
    Column('no_balls_a', Integer),
    Column('penalty_a', Integer),
    Column('extras_a', Integer),
    Column('score_b', Integer),
    Column('wickets_b', Integer),
    Column('overs_played_b', DECIMAL(4, 1)),
    Column('maiden_overs_b', Integer),
    Column('fours_b', Integer),
    Column('sixes_b', Integer),
    Column('boundaries_b', Integer),
    Column('runrate_b', DECIMAL(5, 2)),
    Column('leg_byes_b', Integer),
    Column('byes_b', Integer),
    Column('wides_b', Integer),
    Column('no_balls_b', Integer),
    Column('penalty_b', Integer),
    Column('extras_b', Integer),
    Column('winning_team', String(50)),
    Column('margin', String(50)),
    Column('man_of_the_match', String(100)),
    Column('umpire_1', String(100)),
    Column('umpire_2', String(100)),
    Column('tv_umpire', String(100)),
    Column('match_referee', String(100))
)

# Create table if it doesn't exist
metadata.create_all(engine)

# Insert data into the table
df.to_sql('match_stats', con=engine, if_exists='append', index=False)

# Commit and close connection
connection.close()

print("Data inserted successfully!")


In [ ]:
import pandas as pd
import pymysql
from pymysql import MySQLError

# Define the column names exactly as you provided
columns = [
    'Match ID', 'Match Date', 'Match Time', 'City', 'Stadium', 'Team A', 'Team B', 'Toss Winner', 'Toss Decision', 
    'Score A', 'Wickets A', 'Overs Played A', 'Maiden Overs A', '4s A', '6s A', 'Boundaries A', 'Runrate A', 
    'Leg Byes A', 'Byes A', 'Wides A', 'No Balls A', 'Penalty A', 'Extras A', 'Score B', 'Wickets B', 'Overs Played B', 
    'Maiden Overs B', '4s B', '6s B', 'Boundaries B', 'Runrate B', 'Leg Byes B', 'Byes B', 'Wides B', 'No Balls B', 
    'Penalty B', 'Extras B', 'Wining Team', 'Margin', 'Man of the Match', 'Umpire 1', 'Umpire 2', 'TV Umpire', 'Match Refree'
]

# Load CSV into DataFrame
csv_file_path = 'CWC2023.csv'  # Adjust this path to the correct location of your CSV
df = pd.read_csv(csv_file_path, names=columns, header=0)

# Ensure that date and time columns are properly formatted
df['Match Date'] = pd.to_datetime(df['Match Date'], format='%d-%m-%Y', errors='coerce').dt.date
df['Match Time'] = pd.to_datetime(df['Match Time'], format='%I:%M %p', errors='coerce').dt.time

# Handle invalid date/time by replacing NaT with None (which will be treated as NULL in MySQL)
df['Match Date'] = df['Match Date'].where(df['Match Date'].notna(), None)
df['Match Time'] = df['Match Time'].where(df['Match Time'].notna(), None)


# Establish connection to MySQL using pymysql
try:
    connection = pymysql.connect(
        host='localhost',  # Change to your MySQL host
        user='root',       # Change to your MySQL username
        password='Azsxdcf123@',  # Change to your MySQL password
        database='cricket_data'  # Use the specific database (can also be created first if needed)
    )

    with connection.cursor() as cursor:
        print("Connected to MySQL")

        # Create table if not exists
        create_table_query = """
        CREATE TABLE IF NOT EXISTS match_stats (
            match_id INT PRIMARY KEY,
            match_date DATE,
            match_time TIME,
            city VARCHAR(100),
            stadium VARCHAR(100),
            team_a VARCHAR(50),
            team_b VARCHAR(50),
            toss_winner VARCHAR(50),
            toss_decision VARCHAR(50),
            score_a INT,
            wickets_a INT,
            overs_played_a DECIMAL(4,1),
            maiden_overs_a INT,
            fours_a INT,
            sixes_a INT,
            boundaries_a INT,
            runrate_a DECIMAL(5,2),
            leg_byes_a INT,
            byes_a INT,
            wides_a INT,
            no_balls_a INT,
            penalty_a INT,
            extras_a INT,
            score_b INT,
            wickets_b INT,
            overs_played_b DECIMAL(4,1),
            maiden_overs_b INT,
            fours_b INT,
            sixes_b INT,
            boundaries_b INT,
            runrate_b DECIMAL(5,2),
            leg_byes_b INT,
            byes_b INT,
            wides_b INT,
            no_balls_b INT,
            penalty_b INT,
            extras_b INT,
            winning_team VARCHAR(50),
            margin VARCHAR(50),
            man_of_the_match VARCHAR(100),
            umpire_1 VARCHAR(100),
            umpire_2 VARCHAR(100),
            tv_umpire VARCHAR(100),
            match_referee VARCHAR(100)
        );
        """
        cursor.execute(create_table_query)

        # Insert data into the table
        for i, row in df.iterrows():
            # Ensure match_date and match_time are correctly formatted as strings for SQL
            match_date = row['Match Date'] if pd.notna(row['Match Date']) else None  # Handle NaT
            match_time = row['Match Time'] if pd.notna(row['Match Time']) else None  # Handle NaT

            insert_query = """
            INSERT INTO match_stats (
                match_id, match_date, match_time, city, stadium, team_a, team_b, toss_winner, toss_decision,
                score_a, wickets_a, overs_played_a, maiden_overs_a, fours_a, sixes_a, boundaries_a, runrate_a,
                leg_byes_a, byes_a, wides_a, no_balls_a, penalty_a, extras_a, score_b, wickets_b, overs_played_b,
                maiden_overs_b, fours_b, sixes_b, boundaries_b, runrate_b, leg_byes_b, byes_b, wides_b, no_balls_b,
                penalty_b, extras_b, winning_team, margin, man_of_the_match, umpire_1, umpire_2, tv_umpire, match_referee
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                      %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            """
            cursor.execute(insert_query, tuple(row))

        connection.commit()
        print("Data inserted successfully!")

except MySQLError as e:
    print(f"Error: {e}")
finally:
    if connection.open:
        connection.close()
        print("MySQL connection closed")


In [ ]:
pip install  pymysql
